In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('fourCollections.csv', dtype={'collection_slug': 'str', 'asset_id': 'int', 'asset_name': 'str', 'owner_username': 'str', 'owner_address': 'str', 'event_type': 'str'})
df.drop('Unnamed: 0.1', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('collection_slug', axis=1,inplace=True)
df.drop('asset_contract_date', axis=1, inplace=True)
df.drop('asset_name', axis=1, inplace=True)
df.drop('asset_id', axis=1, inplace=True)
df.drop('owner_username', axis=1, inplace=True)
df.drop('owner_address', axis=1, inplace=True)
df['event_timestamp'] = pd.to_datetime(df['event_timestamp']).dt.strftime('%m/%d/%Y')
df['month'] = pd.to_datetime(df['event_timestamp']).dt.strftime('%m')
df['day'] = pd.to_datetime(df['event_timestamp']).dt.strftime('%d')
df['year'] = pd.to_datetime(df['event_timestamp']).dt.strftime('%Y')
df.drop_duplicates(keep='first')
df

,collection_name,event_timestamp,event_type,month,day,year
0,Azuki,06/17/2022,created,06,17,2022
1,Azuki,06/17/2022,created,06,17,2022
2,Azuki,06/17/2022,created,06,17,2022
3,Azuki,06/17/2022,created,06,17,2022
4,Azuki,06/17/2022,created,06,17,2022
...,...,...,...,...,...,...
1465480,Crypto Coven,06/10/2022,offer_entered,06,10,2022
1465481,Crypto Coven,06/10/2022,offer_entered,06,10,2022
1465482,Crypto Coven,06/10/2022,offer_entered,06,10,2022
1465483,Crypto Coven,06/10/2022,offer_entered,06,10,2022


In [3]:
g = df.groupby(['collection_name', 'month', 'day', 'year']).agg({'event_timestamp': ['count']})

In [4]:
g.drop('Bored Ape Chemistry Club', inplace=True)
g.drop('Bored Ape Kennel Club', inplace=True)
g.drop('DourDarcels', inplace=True)
g.drop('NFTfi Promissory Note', inplace=True)
g.drop('Rug Radio Membership Pass', inplace=True)


In [5]:
g

event_timestamp
                                         count
collection_name month day year                
Azuki           01    12  2022           20659
                      13  2022            7100
                      14  2022            2638
                      15  2022            2292
                      16  2022            1939
...                                        ...
mfers           12    27  2021             509
                      28  2021             601
                      29  2021             931
                      30  2021             812
                      31  2021            1143

[1007 rows x 1 columns]

In [6]:
df = pd.read_csv('g.csv')
df = df[df['year'] == 2022]
df['time'] = pd.to_datetime(df['time'])
df

,collection_name,month,day,year,count,time
0,Azuki,1,12,2022,20659,2022-01-12
1,Azuki,1,13,2022,7100,2022-01-13
2,Azuki,1,14,2022,2638,2022-01-14
3,Azuki,1,15,2022,2292,2022-01-15
4,Azuki,1,16,2022,1939,2022-01-16
...,...,...,...,...,...,...
970,mfers,6,13,2022,913,2022-06-13
971,mfers,6,14,2022,826,2022-06-14
972,mfers,6,15,2022,1289,2022-06-15
973,mfers,6,16,2022,1028,2022-06-16


In [7]:
#dfa = df.groupby([pd.Grouper(key='time', freq='W')])['count'].sum()

In [7]:
# And I need to transform my categorical column (continent) in a numerical value group1->1, group2->2...
df['collection_name']=pd.Categorical(df['collection_name'])

# Set the figure size
plt.figure(figsize=(10, 10))

# Subset of the data for year 1952
data1952 = df[ df.year == 2022 ]

# Scatterplot
plt.scatter(
    x = data1952['day'], 
    y = data1952['month'], 
    s=data1952['count']/5, 
    c=data1952['collection_name'].cat.codes, 
    cmap="Accent", 
    alpha=0.6, 
    edgecolors="white", 
    linewidth=2)
 
# Add titles (main and on axis)
plt.xlabel("Life Expectancy")
plt.ylabel("GDP per Capita")
plt.title("Year 1952")
plt.ylim(0,12)
plt.xlim(0, 34)

(0.0, 34.0)